In [3]:
#!pip install Pymcubes
#!pip install trimesh

In [13]:
import mcubes
import trimesh

In [14]:
import torch
import torch.nn as nn
import numpy as np

import os
import imageio
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader
from new_datasets import data_preprocessing
from model import Voxels,Nerf
from ml_helpers import training
import rendering
import cv2

In [15]:
device = 'cuda'
tn = 2.
tf = 6.
model = torch.load('model_nerf11').to(device)

In [16]:
N = 100
scale = 1.5

x = torch.linspace(-scale, scale, N)
y = torch.linspace(-scale, scale, N)
z = torch.linspace(-scale, scale, N)

x, y, z = torch.meshgrid((x, y, z))

In [17]:
xyz = torch.cat((x.reshape(-1, 1),
                 y.reshape(-1, 1),
                 z.reshape(-1, 1)), dim=1)

In [18]:
with torch.no_grad():
    _, density = model.forward(xyz.to(device), torch.zeros_like(xyz).to(device))
    
density = density.cpu().numpy().reshape(N, N, N)

In [19]:
vertices, triangles = mcubes.marching_cubes(density,  np.mean(density))

In [20]:
mesh = trimesh.Trimesh(vertices / N, triangles)

In [21]:
mesh.show()